***Concept Of OLAP***
* Online analytical processing (OLAP) is a technology that organizes large business databases and supports complex analysis. 
* It can be used to perform complex analytical queries without negatively affecting transactional systems.
* These databases usually have records that are entered one at a time.

### Import Required libraries

In [1]:
import psycopg2
import pandas as pd
import psycopg2.extras
import pandas.io.sql as sqlio

import warnings
warnings.filterwarnings('ignore')

## DataBase Connection

In [2]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "afra1234",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [3]:
con = PostgresConnection().getConnection()

successfully connected to database


In [4]:
conn = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "afra1234",
                                           host = "127.0.0.1",
                                           port = "5432")
print("successfully connected to database")

successfully connected to database


## Simple CUBE query

### Q1: Find the division/district/year/month wise total_sale_price joining fact table and respective dimension table

***Total_sale_price By Division Wise***

In [5]:
conn = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "afra1234",
                                           host = "127.0.0.1",
                                           port = "5432")

query ="SELECT  division, SUM(total_price) total_sales_price FROM ecom_schema.store_dim INNER JOIN ecom_schema.fact_table ON fact_table.store_key = store_dim.store_key GROUP BY store_dim.division;"

data = sqlio.read_sql_query(query,conn)

data

,division,total_sales_price
0,BARISAL,7520343.75
1,CHITTAGONG,19763595.00
2,DHAKA,40764619.75
3,KHULNA,11311610.50
4,RAJSHAHI,12099196.00
5,RANGPUR,8429836.50
6,SYLHET,5512234.25


***Total_sale_price By District Wise***

In [6]:
query ="SELECT  district, SUM(total_price) total_sales_price FROM ecom_schema.store_dim INNER JOIN ecom_schema.fact_table ON fact_table.store_key = store_dim.store_key GROUP BY store_dim.district;"

data = sqlio.read_sql_query(query,conn)

data

,district,total_sales_price
0,BAGERHAT,1307651.00
1,BANDARBAN,995360.75
2,BARGUNA,869041.00
3,BARISAL,2897237.75
4,BHOLA,1012205.75
...,...,...
59,SIRAJGANJ,1295100.75
60,SUNAMGANJ,1593774.00
61,SYLHET,1727717.25
62,TANGAIL,1774650.25


***Total_sale_price By year Wise***

In [7]:
query ="SELECT  year, SUM(total_price) total_sales_price FROM ecom_schema.time_dim INNER JOIN ecom_schema.fact_table ON fact_table.time_key = time_dim.time_key GROUP BY time_dim.year;"

data = sqlio.read_sql_query(query,conn)
data

,year,total_sales_price
0,2014.0,14334731.25
1,2015.0,15095720.25
2,2016.0,14976508.25
3,2017.0,15015806.00
4,2018.0,15108197.25
5,2019.0,14949510.25
6,2020.0,15037190.25
7,2021.0,883772.25


***Total_sale_price By month Wise***

In [8]:
query ="SELECT  month, SUM(total_price) total_sales_price FROM ecom_schema.time_dim INNER JOIN ecom_schema.fact_table ON fact_table.time_key = time_dim.time_key GROUP BY time_dim.month;"

data = sqlio.read_sql_query(query,conn)
data

,month,total_sales_price
0,1.0,9042244.00
1,2.0,8073877.50
2,3.0,8939152.00
3,4.0,8504634.75
4,5.0,9078002.50
5,6.0,8556853.00
6,7.0,9046580.00
7,8.0,8929475.25
8,9.0,8724326.50
9,10.0,8902783.75


### Q2: Find the customer/bank/transaction(cash/online) wise total_sale_price joining fact table and respective dimension table 

In [19]:
query ="SELECT  name, SUM(total_price) Total_sales_price FROM ecom_schema.coustomer_dim INNER JOIN ecom_schema.fact_table ON fact_table.coustomer_key = coustomer_dim.coustomer_key GROUP BY coustomer_dim.coustomer_key;"

data = sqlio.read_sql_query(query,conn)
data

,name,total_sales_price
0,ratmo devi,17104.50
1,pooja devi,9084.25
2,indra megwal,11298.50
3,rashika,13431.50
4,afsana,14034.25
...,...,...
9186,beauty dass,11033.00
9187,mulina,12721.00
9188,raju garg,13633.50
9189,reena,10094.25


In [9]:
query ="SELECT  division, total_price FROM ecom_schema.store_dim INNER JOIN ecom_schema.fact_table ON fact_table.store_key = store_dim.store_key ORDER BY store_dim.division;"

data = sqlio.read_sql_query(query,conn)
data

,division,total_price
0,BARISAL,120.00
1,BARISAL,84.00
2,BARISAL,102.00
3,BARISAL,85.00
4,BARISAL,33.75
...,...,...
999995,SYLHET,220.00
999996,SYLHET,72.00
999997,SYLHET,60.00
999998,SYLHET,16.00
